In [58]:
from IPython.display import Image
from IPython.core.display import HTML 

import warnings
warnings.filterwarnings('ignore')

## [DSO] Machine Learning Seminar Vol.8
2020-09-xx  
SKUE

In [59]:
Image(url="https://image.yodobashi.com/product/100/000/009/002/935/105/100000009002935105_10204.jpg")

# 機械学習の適用2 - Webアプリケーション
+ 訓練済みの機械学習モデルの保存
+ SQLiteの活用
+ Flaskを用いたWebアプリケーションの開発
+ デプロイ

## 学習済みのscikit-learnの推定器をシリアライズする

第8章で扱ったアウトオブコア学習の結果をpickleで保存しておく。

In [1]:
# import pickle
# import os
# dest = os.path.join('movieclassfier', 'pkl_objects')

# if not os.path.exists(dest):
#     os.makedirs(dest)
    
# pickle.dump(stop,
#                        open(os.path.join(dest, 'stopwords.pkl'), 'wb'),
#                        protocol=4)

# pickle.dump(clf,
#                        open(os.path.join(dest, 'classifier.pkl'), 'wb'),
#                        protocol=4)

%%file  
を使えばjupyter notebookで.pyファイルを生成できる。

In [2]:
%%file movieclassfier/vectorizer.py

from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(os.path.join(cur_dir,
                                            'pkl_objects',
                                            'stopwords.pkl'), 'rb'))

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + '  '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop ]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                                            n_features=2**21,
                                            preprocessor=None,
                                            tokenizer=tokenizer)

Writing movieclassfier/vectorizer.py


以下はmovieclassfierのディレクトリ下でpythonを立ち上げて実行

In [3]:
# import pickle
# import re
# import os
# from vectorizer import vect
# clf = pickle.load(open(os.path.join('pkl_objects', 'classifier.pkl'), 'rb'))


# import numpy as np
# label = {0:'negative',
#               1:'positive'}

# example = ['I love this movie']
# X = vect.transform(example)
# print('Prediction: %s\nProbability: %.2f%%' \
#       %(label[clf.predict(X)[0]],np.max(clf.predict_proba(X))*100))

## データストレージとしてSQLiteデータベースを設定する
+ SQLiteとは
 + OSSのSQLデータベース
 + サーバは用意しなくてもいい。
 + 全てが1つのファイルに含まれたデータベースと考えることができる。

テーブルを生成して、レコードを2件追加する。

In [7]:
import sqlite3
import os

if os.path.exists('reviews.sqlite'):
    os.remove('reviews.sqlite')
    
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE review_db (review TEXT, sentiment INTEGER, date TEXT) ')
example1 = 'I love this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES"\
                     "(?, ?, DATETIME('now'))", (example1, 1))

example2 = 'I disliked this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES"\
                     "(?, ?, DATETIME('now'))", (example2, 0))
conn.commit()
conn.close()

SQLiteに保存したデータを読み込む

In [8]:
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute("SELECT * FROM review_db WHERE date"\
                   " BETWEEN '2017-01-01 00:00:00' AND DATETIME('now') ")
results = c.fetchall()
conn.close()
print(results)

[('I love this movie', 1, '2020-09-19 08:27:24'), ('I disliked this movie', 0, '2020-09-19 08:27:24')]


## Flaskを使ってWebアプリケーションを開発する
+ Flaskとは
 + 軽量なWebアプリケーションのフレームワーク

In [10]:
# !pip install flask

In [12]:
# !mkdir 1st_flask_app_1

In [14]:
# !mkdir 1st_flask_app_1/templates

In [15]:
%%file 1st_flask_app_1/app.py

from flask import Flask, render_template

app = Flask(__name__) # 初期化して、テンプレが同じディレクトリ内で見つかるようにしている。
@app.route('/') # インデックス関数が実行すべきURLを指定している。

def index():
    return render_template('first_app.html') # レンダリング

if __name__ == '__main__':
    app.run() # サーバ上のアプリケーションを実行

Writing 1st_flask_app_1/app.py


In [17]:
%%file 1st_flask_app_1/templates/first_app.html
<!doctype html>
<html>
    <head>
        <title>First app</title>
    </head>
        <div>Hi, this is my first Flaskweb app!</div>
    </body>
</html>

Overwriting 1st_flask_app_1/templates/first_app.html


In [18]:
# %%file 1st_flask_app_1/templates/first_app.html
# <!doctype html>
# <html>
#     <head>
#         <title>First app</title>
#     </head>
#         A4用紙1枚にまとめるイメージでメモを残そうという取り組み。

# 今回は機械学習でとりわけ、回帰の問題を扱うことが仕事であった際に参照できるメモの一つとして残したい。
# 先日、<a target="_blank" href="https://github.com/catboost/catboost/releases/tag/v0.24.1" rel="noopener noreferrer">Catboostのドキュメント</a>に参考文献として<a target="_blank" href="https://arxiv.org/pdf/2006.10562.pdf" rel="noopener noreferrer">Uncertainty in Gradient Boosting via Ensembles</a>の存在を知った。業務において、価格の予測とか、期間の予測とかマーケティングのデータでも良く扱うものなので、分類だけでなく回帰についてもバランス良く知見を貯めたいところ。
# <h2>目次</h2>
# <a href="#anchor1">Abstract</a>
# <a href="#anchor2">Introduction</a>
# <a href="#anchor3">Background and related work</a>
# <a href="#anchor4">Generating ensembles of GDBT models</a>
# <a href="#anchor5">Experiments on Synthetic Data</a>
# <a href="#anchor6">Experiments on classfication and regression datasets</a>
# <a href="#anchor7">Conclusion</a>
# <a href="#anchor8">Broader Impact</a>
# <h2><a id="anchor1">Abstract</a></h2>
# 機械学習のモデルは予測において点推定しかしないが、実務上は不確実性を定量化したいニーズがある。この研究では、予測の不確実性推定値を導出するためのアンサンブルベースの枠組みが紹介されている。予測における不確実性は「データに起因する複雑さとノイズの不確実性」「特徴空間において、情報がないことに起因する不確実性」に分解される。
# 手法としては確率的勾配ブースティングと確率的勾配ランジュバンブースティング(SGLB)が扱われている。SGLBは一つの勾配ブースティングモデルからのアンサンブルを可能にし、モデルの複雑さを減らすことができる。また、予測性能の向上について様々なデータセットを使って示されている。
# <h2><a id="anchor2">Introduction</a></h2>
# 一般的な勾配ブースティングについて、回帰の際に点推定しか返さないことを挙げている。NGBoostについても言及している。NGBoostなら、勾配ブースト決定木(GBDT)を訓練して平均値を返せるとのこと。ただし、データの不確実性に起因する不確実性しか捕捉できないという問題がある。モデルに関する知識の欠落によって生じる不確実性に関して扱うことはできない。近年ではニューラルネットワークを用いて知識を起因とする不確実性の研究がなされているらしい。今回扱う、アンサンブルアプローチは全体の不確実性を「データの不確実性」と「知識の不確実性」に分解する。扱うモデルは確率的勾配ブースティングと、確率的勾配ランジュバンブースティング(SGLB)となっており、SGLBの利点として、漸近的に真のベイジアン事後分布からサンプリングことが挙げられている。また、SGLBによって1つの勾配ブースティングのみをモデルを使ってアンサンブルをすることができるとされている。これにより計算の複雑さを減らすことができる。結果として、予測性能が高まり、誤りやドメインの外側のデータの入力の検知ができるようになったとされている。
# <h2><a id="anchor3">Background and related work</a></h2>
# <ul>
#  	<li>ベイジアンのアプローチ
# ベイズ統計学を使っている。予測の多様性について考察することで、知識の不確実性の推定を行うことができるとしている。さらに、確率モデルのアンサンブルを行なっている。アンサンブルモデルの不一致さ、広がりの程度が知識の不確実性につながるとしている。事後分布の近似のための研究が色々となされ、ニューラルネットワークのモデルなども使われた。モデルの事後分布をもとに、予測の事後分布を手に入れる。予測の事後分布のエントロピーが、予測の不確実性を推定する。知識の不確実性は、全ての不確実性からデータの不確実性の期待値を差し引いた相互情報量で表される。確率的な回帰モデルのアンサンブルを考える時、予測事後分布のエントロピーと相互情報量を推定することができない。代わりに総変動の法則に従ってなら推定することができるが、1~2次のモーメントに基づいているだけなので、高次元での不確実性に関する見落としが起きうる。</li>
#  	<li>勾配ブースティング
# 弱学習器は浅い決定木によって形成される。ただ、勾配ブースティングの回帰モデルは点での予測しかしない。予測の不確実性に関する研究はほとんどされていない。近年、NGBoost (Natural Gradient Boosting)が登場し、予測の不確かさを推定することができるようになった。NGBoostは入力変数xが与えられたもとでの、出力変数の条件付き事後分布の推定を行ってくれる。</li>
# </ul>
# <h2><a id="anchor4">Generating ensembles of GDBT models</a></h2>
# NGBoostはデータの不確実性を捉えることができるものの、知識の不確実性については捉えることができない。
# 知識の不確実性を捉えるための方法としては、事後分布からサンプリングされた予測モデル(GBDT)のアンサンブルが挙げられる。
# そもそもGBDT自身が木のアンサンブルであるが、そのGBDTのアンサンブルを考える。

# アンサンブルをする方法としては、確率的勾配ブースティングのアプローチで生成された独立なモデルを扱うことがあげられている。
# もとのモデルにノイズを注入して、出力をぶれさせることで予測の分布を得ようとしている。ただ、これは事後分布の推定を保証しないらしい。

# ただし、確率的勾配ランジュバンブースティング(SGLB)アルゴリズムを用いることで真の事後分布からGBDTモデルをサンプリングすることができるらしい。
# 非凸の損失関数においての最適化の収束のために、確率的勾配ランジュバン力学が用いられているとのこと。ランジュバン力学ってなんだろうか。

# 確率的勾配ブースティングとの違いは2つある。
# <ul>
#  	<li>1.ガウシアンノイズが勾配に明示的に加えられている。
# <ul>
#  	<li>正規分布の平均は0で分散に逆拡散温度というものがある。ノイズをコントロールするものであることに間違いはない。確かに物理っぽい。STANのリープフロッグとかを思い出した。このノイズが最適値を見つけるのに役立つらしい。</li>
# </ul>
# </li>
#  	<li>2.ブースティングモデルの更新式に正則化パラメータを設けている。
# <ul>
#  	<li>定常分布に弱く収束していくマルコフ連鎖にパラメータが従うらしい。</li>
# </ul>
# </li>
# </ul>
# 対数尤度に正則化項が残るものの、比例するらしく、真の事後分布に従うと見なせるらしい。

# 確率的勾配ブースティングや確率的勾配ランジュバンブースティングは計算する上で重大なオーバーヘッドが生じるらしくイタレーションを減らすなど質を下げたり複雑さを増したりする必要があるとのこと。それを避けるために、仮想SGLBアンサンブルというものが提案されている。

# SGLBアルゴリズムのイタレーションごとのパラメータは弱く定常に収束するマルコフ連鎖に従うことから、それらをモデルのアンサンブルと見なす。
# しかしながらモデル同士が強く相関してしまうので、アンサンブルの質が下がる。ただし、各々のパラメータのK番目の集合のみを使うことで克服できるとある。
# K番目のパラメータ集合のみのモデルで構築したアンサンブルを仮想アンサンブルと呼んでいる。
# <h2><a id="anchor5">Experiments on Synthetic Data</a></h2>
# ここではGBDTモデルのアンサンブルアルゴリズムを合成データを用いて実験している。
# 合成データとしては、以下の2つの1次元データセットが扱われている。
# <ul>
#  	<li>合成データセット
# <ul>
#  	<li>分散が増加していく絶対値x</li>
#  	<li>分散が減少していく絶対値x</li>
# </ul>
# </li>
# </ul>
# <ul>
#  	<li>比較するモデル
# <ul>
#  	<li>SGLB</li>
#  	<li>SGB</li>
#  	<li>vSGLB（仮想SGLB）：SGLBから生成</li>
# </ul>
# </li>
# </ul>
# <ul>
#  	<li>結果
# <ul>
#  	<li>分散が減少していく絶対値xのケースだと、データの不確実性に関して、SGBではサンプルの領域外で過剰に不確実性が増してしまう。</li>
#  	<li>分散が増加していく絶対値xのケースだと、知識の不確実性に関して、SGBではサンプルの領域外で不確実性が増してしまう。</li>
# </ul>
# </li>
# </ul>
# サンプルの領域外のほうが、領域内よりも不確実性が大きいはずだが、実験データではそうではないらしい。
# <h2><a id="anchor6">Experiments on classfication and regression datasets</a></h2>
# GBDTモデルのアンサンブルアルゴリズムを色々なデータのいろいろなタスク（回帰、分類）で予測性能をみている。
# <ul>
#  	<li>タスク
# <ul>
#  	<li>回帰タスク</li>
#  	<li>分類タスク</li>
# </ul>
# </li>
# </ul>
# <ul>
#  	<li>データセット
# <ul>
#  	<li>18種類くらいある。（Wineとか、解約のデータとかAmazonのデータとかいろいろ）</li>
# </ul>
# </li>
# </ul>
# <ul>
#  	<li>比較するモデル
# <ul>
#  	<li>SGB：このアンサンブルは10個の独立したモデルで、それぞれ1K本の木を持つ。</li>
#  	<li>SGLB：このアンサンブルは10個の独立したモデルで、それぞれ1K本の木を持つ。</li>
#  	<li>vSGLB（仮想SGLB）：[500, 1000]の区間から50番目のモデルをアンサンブルに追加していっている。</li>
# </ul>
# </li>
# </ul>
# <ul>
#  	<li>どう実装しているか
# <ul>
#  	<li>CatBoostライブラリをベース</li>
#  	<li>分類では二値分類のクラスの確率分布を生成</li>
#  	<li>回帰では正規分布の平均と分散を生成</li>
#  	<li>負の対数尤度を最適化している</li>
#  	<li>ハイパーパラメータはランダムサーチ</li>
# </ul>
# </li>
# </ul>
# <ul>
#  	<li>何の指標を見ているか
# <ul>
#  	<li>負の対数尤度</li>
#  	<li>誤分類率</li>
#  	<li>RMSE</li>
#  	<li>不確実性の合計、知識の不確実性：Prediction-Rejection Ratio(PPR)というもので測る。どれくらい不確実性が誤差と順番と相関するかを測るらしい。100が最大で、ランダムだと0になる。</li>
#  	<li>AUC：領域外のデータに関する評価に用いる。領域外データは合成して生成した。</li>
# </ul>
# </li>
# </ul>
# <ul>
# <ul>
#  	<li>分類：負の対数尤度（NLL）
# <ul>
#  	<li>アンサンブル手法に軍配が上がるが、違いは軽微である。</li>
#  	<li>仮想アンサンブルが勝るときがある。</li>
# </ul>
# </li>
#  	<li>分類：誤分類率
# <ul>
#  	<li>アンサンブル手法に軍配が上がるが、違いは軽微である。</li>
# </ul>
# </li>
#  	<li>回帰：RMSE
# <ul>
#  	<li>アンサンブル手法に軍配が上がるが、違いは軽微である。</li>
# </ul>
# </li>
# </ul>
# <ul>
#  	<li>SGBやSGLBはパフォーマンスが良い。
# <ul></ul>
# </li>
#  	<li>vSGLBはわずかにパフォーマンスが悪い。自己相関があるなかでアンサンブルしているためパフォーマンス悪化が生じていると考えられる。
# <ul>
#  	<li>ただ単一モデルのものよりかは良い。</li>
#  	<li>コストをかけずに精度を上げたいという観点ではvSGLBはよい選択肢となる。</li>
# </ul>
# </li>
# </ul>
# </ul>
# <h2><a id="anchor7">Conclusion</a></h2>
# この論文では、GBDTを用いたアンサンブルベースの不確実性推定モデルが扱われた。SGB、SGLB、仮想SGLBについて合成データや、分類や回帰のタスクにおいて比較した結果、アンサンブル手法が精度面で秀でているわけではないことが示された。
# しかし、知識の不確実性（モデルで十分に想定できていない不確実性）を扱う際にアンサンブル手法が役立つことがわかった。
# 知識の不確実性を扱う場合に、計算コストの少ない手法として仮想のSGLBが提案され、精度において十分ではないが、低コストで知識の不確実性を扱う観点において有用であることが示された。
# <h2><a id="anchor8">Broader Impact</a></h2>
# <ul>
#  	<li>不確実性の推定は、機械学習モデルをより安全に、より信頼できるものにするという観点で重要。</li>
#  	<li>GBDT系の手法はニューラルネットワーク系の手法よりも計算コストが低くて良い。</li>
#  	<li>不確実性の推定が貧弱なものとならないように引き続き研究が求められる。</li>
# </ul>
# <h2>追記</h2>
# 不確実性について扱われた、ソースコードの載っている記事を集める。
# <ul>
#  	<li><a target="_blank" href="https://brendanhasz.github.io/2018/12/15/quantile-regression" rel="noopener noreferrer">Prediction Intervals for Taxi Fares using Quantile Loss</a></li>
# </ul>
#     </body>
# </html>

In [20]:
# !pip install wtforms

In [21]:
# !mkdir 1st_flask_app_2

In [23]:
# !mkdir 1st_flask_app_2/static

In [24]:
# !mkdir 1st_flask_app_2/templates

In [30]:
%%file 1st_flask_app_2/app.py

from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators

app = Flask(__name__)

class HelloForm(Form):
    sayhello = TextAreaField(' ', [validators.DataRequired()]) # テキストフィールドの拡張
    
@app.route('/')
def index():
    form = HelloForm(request.form)
    return render_template('first_app.html', form=form)

@app.route('/hello', methods=['POST'])
def hello():
    form = HelloForm(request.form)
    if request.method == 'POST' and form.validate(): # POSTメソッドを使ってフォームのデータをサーバに送信する。
        name = request.form['sayhello']
        return render_template('hello.html', name=name)
    
    return render_template('first_app.html', form=form)

if __name__ == '__main__':
    app.run(debug=True)

Overwriting 1st_flask_app_2/app.py


In [35]:
%%file 1st_flask_app_2/templates/_formhelpers.html

{% macro render_field(field) %}
    <dt>{{ field.label  }}
    <dd>{{ field(**kwargs) | safe }}
    {% if field.errors %}
        <ul class=errors>
        {% for error in field.errors %}
            <li>{{  error  }}</li>
        {% endfor %}
        </ul>
    {% endif %}
    </dd>
    </dt>
{% endmacro %}

Writing 1st_flask_app_2/templates/_formhelpers.html


In [36]:
%%file 1st_flask_app_2/static/style.css

body{
    font-size: 2em;
}

Overwriting 1st_flask_app_2/static/style.css


In [37]:
%%file 1st_flask_app_2/templates/first_app.html
<!doctype html>
<html>
    <head>
        <titile>First app</title>
        <link rel="stylesheet"
                    href="{{ url_for('static', filename='style.css')}}">
    </head>
    <body>
    {% from "_formhelpers.html" import render_field %}
    
    <div>Whats your name?</div>
    <form method=post action="/hello">
        <dl>
            {{ render_field(form.sayhello) }}
        </dl>
        <input type=submit value='Say Hello' name='submit_btn'>
    </form>
    </body>
</html>

Overwriting 1st_flask_app_2/templates/first_app.html


In [38]:
%%file 1st_flask_app_2/templates/hello.html
<!doctype html>
<html>
    <head>
        <titile>First app</title>
        <link rel="stylesheet"
                    href="{{ url_for('static', filename='style.css')}}">
    </head>
    <body>
        <div>Hello {{ name }}</div>
    </body>
</html>

Overwriting 1st_flask_app_2/templates/hello.html


In [57]:
%%file movieclassfier/app.py

from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators
import pickle
import sqlite3
import os
import numpy as np

# ローカルディレクトリからHashingVectorizerをインポート
from vectorizer import vect

# ローカルディレクトリからupdate関数をインポート
from update import update_model

app = Flask(__name__)

####### 分類器の準備
cur_dir = os.path.dirname(__file__)
clf = pickle.load(open(os.path.join(cur_dir, 'pkl_objects', 'classifier.pkl'), 'rb'))
db = os.path.join(cur_dir, 'reviews.sqlite')

def classify(document):
    label = {0: 'negative',
                  1: 'positive'}
    X = vect.transform([document])
    y = clf.predict(X)[0]
    proba = clf.predict_proba(X).max()
    return label[y], proba

def train(document, y):
    X = vect.transform([document])
    clf.partial_fit(X, [y])
    
def sqlite_entry(path, document, y):
    conn = sqlite3.connect(path)
    c = conn.cursor()
    c.execute("INSERT INTO review_db (review, sentiment, date)"\
                        " VALUES (?, ?, DATETIME('now'))", (document, y))
    conn.commit()
    conn.close()
    
    
###### Flask
class ReviewForm(Form):
    moviereview = TextAreaField(' ',
                                                           [validators.DataRequired(),
                                                               validators.length(min=15)]) # レビューは15文字以内
    
@app.route('/')
def index():
    form = ReviewForm(request.form)
    return render_template('reviewform.html', form=form)

@app.route('/requests', methods=['POST'])
def results():
    form = ReviewForm(request.form)
    if request.method == 'POST' and form.validate():
        review = request.form['moviereview']
        y, proba = classify(review)
        return render_template('results.html',
                                                  contents=review,
                                                  prediction=y,
                                                  probability=round(proba*100, 2))
    return render_template('reviewform.html', form=form)

@app.route('/thanks', methods=['POST'])
def feedback():
    feedback = request.form['feedback_button']
    review = request.form['review']
    prediction = request.form['prediction']
    
    inv_label = {'negative':0,
                          'positive':1}
    y = inv_label[prediction]
    if feedback == 'Incorrect':
        y = int(not(y))
    train(review, y)
    sqlite_entry(db, review, y)
    return render_template('thanks.html')

if __name__ == '__main__':
    clf = update_model(db_path=db, model=clf, batch_size=10000)
    app.run(debug=True)

Overwriting movieclassfier/app.py


In [43]:
# !mkdir movieclassfier/templates

In [50]:
%%file movieclassfier/templates/reviewform.html
<!doctype html>
<html>
    <head>
        <link rel="stylesheet"
                    href="{{ url_for('static', filename='style.css')}}">
    </head>
    <body>
        <h2>Please enter your movie review:</h2>
        
        {% from "_formhelpers.html" import render_field %}
        
        <form method=post action="/requests">
            <dl>
                {{  render_field(form.moviereview, cols='30', rows='10') }}
            </dl>
            <div>
                <input type=submit value='Submit review' name='submit_btn'>
            </div>
        </form>
    </body>
</html>

Overwriting movieclassfier/templates/reviewform.html


In [49]:
%%file movieclassfier/templates/results.html
<!doctype html>
<html>
    <head>
        <titile>Movie Classification</title>
        <link rel="stylesheet"
                    href="{{ url_for('static', filename='style.css')}}">
    </head>
    <body>
        <h3>Your movie review:</h3>
            <div>{{ content }}</div>
            
        <h3>Prediction</h3>
            <div>This movie review is <strong>{{ prediction }}</strong>
                (probability: {{ probability }}%).</div>
                
            <div id='button'>
                <form action="/thanks" method="post">
                    <input type=submit value='Correct' name='feedback_button'>
                    <input type=submit value='Incorrect' name='feedback_button'>
                    <input type=hidden value='{{ prediction }}' name='prediction'>
                    <input type=hidden value='{{ content }}' name='review'>
                </form>
            </div>
            
            <div id='button'>
                <form action="/">
                    <input type=submit value='Submit another review'>
                </form>
            </div>
    </body>
</html>

Overwriting movieclassfier/templates/results.html


In [47]:
# !mkdir movieclassfier/static

In [48]:
%%file movieclassfier/static/style.css

body{
    width:600px;
}
.button{
    padding-top: 20px;
}

Writing movieclassfier/static/style.css


In [51]:
%%file movieclassfier/templates/thanks.html
<!doctype html>
<html>
    <head>
        <titile>Movie Classification</title>
        <link rel="stylesheet"
                    href="{{ url_for('static', filename='style.css')}}">
    </head>
    <body>
        <h3>Thank you for your feedbacks!</h3>
            <div id='button'>
                <form action="/">
                    <input type=submit value='Submit another review'>
                </form>
            </div>
    </body>
</html>

Writing movieclassfier/templates/thanks.html


In [52]:
%%file movieclassfier/templates/_formhelpers.html

{% macro render_field(field) %}
    <dt>{{ field.label  }}
    <dd>{{ field(**kwargs) | safe }}
    {% if field.errors %}
        <ul class=errors>
        {% for error in field.errors %}
            <li>{{  error  }}</li>
        {% endfor %}
        </ul>
    {% endif %}
    </dd>
    </dt>
{% endmacro %}

Writing movieclassfier/templates/_formhelpers.html


/home/kamonohashiperio/movieclassifier/app.py  

virtualenvでsklearnを入れないと動かない。

mkvirtualenv periove --python=/usr/bin/python3.8

https://www.pythonanywhere.com/forums/topic/14032/

In [56]:
%%file movieclassfier/update.py

import pickle
import sqlite3
import numpy as np
import os

# ローカルディレクトリからHashingVectorizerをインポート
from vectorizer import vect

def update_model(db_path, model, batch_size=10000):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute('SELECT * from review_db')
    
    results = c.fetchmany(batch_size)
    while results:
        data = np.array(results)
        X = data[:, 0]
        y = data[:, 1].astype(int)
        classes = np.array([0, 1])
        X_train = vect.transform(X)
        model.partial_fit(X_train, y, classes=classes)
        results = c.fetchmany(batch_size)
        
    conn.close()
    return model

cur_dir = os.path.dirname(__file__)
clf = pickle.load(open(os.path.join(cur_dir, 'pkl_objects', 'classifier.pkl'), 'rb'))
db = os.path.join(cur_dir, 'reviews.sqlite')

clf = update_model(db_path=db, model=clf, batch_size=10000)

# classifier.pklのファイルを永続的に更新したい場合は以下のコードのコメントを解除
# pickle.dump(clf, open(os.path.join(cur_dir, 'pkl_objects', 'classifier.pkl'), 'wb'), protocol=4)

Overwriting movieclassfier/update.py
